<a href="https://colab.research.google.com/github/anweshaban21/DeepSectoralAI/blob/main/GRU_LSTM_BiLstm_HybridwithGARCH_Beta_SectoralAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.3/978.3 kB 13.2 MB/s eta 0:00:00


In [2]:
pip install yfinance pandas numpy matplotlib


In [3]:
# Define your sectors and representative stocks
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start_date = "2015-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


Fetching data for IT sector...


/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Pharma sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


handling missing data


In [5]:
for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()  # Forward and backward fill
    sector_data[sector] = df


Calculating Log Returns

In [6]:
import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [ ]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")


In [ ]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2015-01-01,486.962860,436.521210,294.023865,624.794739
2015-01-02,497.853119,442.596436,302.369690,631.223450
2015-01-05,500.999176,438.859589,302.995605,628.691589
2015-01-06,483.090820,432.027863,290.142975,621.195740
2015-01-07,482.703583,433.288757,282.297974,631.198608
...,...,...,...,...
2025-08-18,1082.599976,2003.599976,1434.599976,2001.400024
2025-08-19,1083.199951,1991.099976,1436.300049,2029.900024
2025-08-20,1080.199951,1988.199951,1430.599976,2017.500000


In [ ]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2015-01-02,0.022117,0.013821,0.027990,0.010237
2015-01-05,0.006299,-0.008479,0.002068,-0.004019
2015-01-06,-0.036400,-0.015690,-0.043344,-0.011995
2015-01-07,-0.000802,0.002914,-0.027411,0.015974
2015-01-08,0.006596,0.020788,0.026835,0.052957
...,...,...,...,...
2025-08-18,0.013391,0.006258,0.005101,0.011660
2025-08-19,0.000554,-0.006258,0.001184,0.014140
2025-08-20,-0.002773,-0.001458,-0.003976,-0.006127


In [7]:
pip install arch


In [8]:
from arch import arch_model
import pandas as pd
import numpy as np


In [9]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility


In [10]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols


Estimating GARCH volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002847. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002803. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Banking sector...
Estimating GARCH volatility for Pharma sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002817. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003323. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Auto sector...
Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.000217. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001976. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarni

In [ ]:
print(garch_volatility_data['FMCG'].tail())


            DABUR.NS  HINDUNILVR.NS    ITC.NS
Date                                         
2025-08-18  0.012895       0.011965  0.011755
2025-08-19  0.016712       0.015832  0.012331
2025-08-20  0.015747       0.015455  0.012265
2025-08-21  0.016680       0.016398  0.012296
2025-08-22  0.019554       0.015574  0.012076


LSTM

In [11]:
pip install numpy pandas scikit-learn tensorflow


In [12]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler


In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [33]:

def predict_lstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_lstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    # Estimate volatility as rolling std of predicted returns
    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [34]:
lstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting LSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_lstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    lstm_volatility_data[sector] = sector_vols


Predicting LSTM volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


...

82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [37]:
print(lstm_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2025-08-18    0.001528  0.000934  0.000789  0.000486
2025-08-19    0.001543  0.000882  0.000792  0.000499
2025-08-20    0.001530  0.000839  0.000781  0.000519
2025-08-21    0.001517  0.000929  0.000926  0.000584
2025-08-22    0.001519  0.001125  0.001061  0.000671


In [38]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional


def build_bilstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=50, return_sequences=False), input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model




In [39]:
def predict_bilstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_bilstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [ ]:

bilstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting BiLSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_bilstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    bilstm_volatility_data[sector] = sector_vols


Predicting BiLSTM volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


...

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
print(bilstm_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2025-08-18    0.001100  0.001233  0.000705  0.000863
2025-08-19    0.001133  0.001196  0.000707  0.000876
2025-08-20    0.001178  0.001105  0.000717  0.000824
2025-08-21    0.001245  0.000995  0.000765  0.000803
2025-08-22    0.001279  0.000919  0.000794  0.000812


In [42]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import GRU, Dense



def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model





In [43]:
def predict_gru_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)  # same prepare function works
    model = build_gru_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [ ]:
gru_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting GRU volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_gru_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    gru_volatility_data[sector] = sector_vols


Predicting GRU volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


...

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Predicting GRU volatility for Auto sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [44]:
print(gru_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2025-08-18    0.001181  0.000888  0.000377  0.000481
2025-08-19    0.001223  0.000848  0.000382  0.000502
2025-08-20    0.001264  0.000812  0.000383  0.000530
2025-08-21    0.001260  0.000849  0.000441  0.000595
2025-08-22    0.001242  0.001051  0.000484  0.000677


In [45]:
gru_volatility_data['IT'].dropna

<bound method DataFrame.dropna of             HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2015-02-02         NaN       NaN       NaN       NaN
2015-02-03         NaN       NaN       NaN       NaN
2015-02-04         NaN       NaN       NaN       NaN
2015-02-05         NaN       NaN       NaN       NaN
2015-02-06         NaN       NaN       NaN       NaN
...                ...       ...       ...       ...
2025-08-18    0.001181  0.000888  0.000377  0.000481
2025-08-19    0.001223  0.000848  0.000382  0.000502
2025-08-20    0.001264  0.000812  0.000383  0.000530
2025-08-21    0.001260  0.000849  0.000441  0.000595
2025-08-22    0.001242  0.001051  0.000484  0.000677

[2607 rows x 4 columns]>

In [47]:
gru_volatility_data['IT'].tail()

,HCLTECH.NS,INFY.NS,TCS.NS,WIPRO.NS
Date,,,,
2025-08-18,0.001181,0.000888,0.000377,0.000481
2025-08-19,0.001223,0.000848,0.000382,0.000502
2025-08-20,0.001264,0.000812,0.000383,0.000530
2025-08-21,0.001260,0.000849,0.000441,0.000595
2025-08-22,0.001242,0.001051,0.000484,0.000677


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']


In [ ]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta



In [ ]:
beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas


In [ ]:
beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


Computing Sharpe Ratio

In [ ]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes


In [48]:
features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            lstm_mean = lstm_volatility_data[sector][stock].mean()
            bilstm_mean = bilstm_volatility_data[sector][stock].mean()
            gru_mean = gru_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]

            features.append([sector, stock, beta, garch_mean, lstm_mean, bilstm_mean, gru_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")

features_df = pd.DataFrame(
    features,
    columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'LSTM_Vol', 'BiLSTM_Vol', 'GRU_Vol', 'Sharpe']
)


In [49]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol','LSTM_Vol', 'BiLSTM_Vol', 'GRU_Vol', 'Sharpe'])


In [50]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [  0.01639802  -2.77465893   7.54076869 -13.68621258]
Intercept: 0.06700001873206124


In [51]:
X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

# Drop rows with NaNs
df_clean = pd.concat([X, y], axis=1).dropna()

X = df_clean[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = df_clean['Sharpe']

model = LinearRegression().fit(X, y)
print("Learned Weights:", model.coef_)
print("Intercept:", model.intercept_)


Learned Weights: [  0.01639802  -2.77465893   7.54076869 -13.68621258]
Intercept: 0.06700001873206124


In [52]:
features_df['Risk_Score'] = X @ weights


In [53]:
for sector in features_df['Sector'].unique():
    print(f"\nTop 3 safest stocks in {sector}:")
    print(features_df[features_df['Sector'] == sector].sort_values('Risk_Score').head(3)[['Stock', 'Risk_Score']])



Top 3 safest stocks in IT:
        Stock  Risk_Score
0  HCLTECH.NS   -0.042905
2      TCS.NS   -0.041395
1     INFY.NS   -0.039796

Top 3 safest stocks in Banking:
          Stock  Risk_Score
4   AXISBANK.NS   -0.044873
6  ICICIBANK.NS   -0.037768
7  KOTAKBANK.NS   -0.034003

Top 3 safest stocks in Pharma:
           Stock  Risk_Score
10  SUNPHARMA.NS   -0.049863
9     DRREDDY.NS   -0.049455
8       CIPLA.NS   -0.042737

Top 3 safest stocks in Auto:
            Stock  Risk_Score
13  TATAMOTORS.NS   -0.060673
11         M&M.NS   -0.041959
12      MARUTI.NS   -0.038221

Top 3 safest stocks in FMCG:
            Stock  Risk_Score
16         ITC.NS   -0.040178
14       DABUR.NS   -0.037521
15  HINDUNILVR.NS   -0.032061


GRU Bilstm

In [54]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','BiLSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [  0.02309449  -3.3599256    3.87533145 -10.75128331]
Intercept: 0.07058965620825018


In [57]:
features_df['Risk_Score_GRU_Bi_Beta_GARCH'] = X @ weights


In [58]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (beta + GARCH + GRU+ bilstm):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_Bi_Beta_GARCH')
          .head(3)[['Stock', 'Risk_Score_GRU_Bi_Beta_GARCH']])



Safest stocks in IT (beta + GARCH + GRU+ bilstm):
        Stock  Risk_Score_GRU_Bi_Beta_GARCH
0  HCLTECH.NS                     -0.047495
3    WIPRO.NS                     -0.044187
1     INFY.NS                     -0.043281

Safest stocks in Banking (beta + GARCH + GRU+ bilstm):
          Stock  Risk_Score_GRU_Bi_Beta_GARCH
4   AXISBANK.NS                     -0.049435
7  KOTAKBANK.NS                     -0.041148
6  ICICIBANK.NS                     -0.041113

Safest stocks in Pharma (beta + GARCH + GRU+ bilstm):
           Stock  Risk_Score_GRU_Bi_Beta_GARCH
10  SUNPHARMA.NS                     -0.052978
9     DRREDDY.NS                     -0.052067
8       CIPLA.NS                     -0.045685

Safest stocks in Auto (beta + GARCH + GRU+ bilstm):
            Stock  Risk_Score_GRU_Bi_Beta_GARCH
13  TATAMOTORS.NS                     -0.062717
11         M&M.NS                     -0.047246
12      MARUTI.NS                     -0.040604

Safest stocks in FMCG (beta + GARCH + GRU+ b

LSTM + BiLSTM + GARCH + BETA

In [59]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'LSTM_Vol','BiLSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [ 0.0177635  -2.55574718 -7.87868108 -3.12607329]
Intercept: 0.06702627899020674


In [60]:
features_df['Risk_Score_lstm_bi_garch_Beta'] = X @ weights


In [61]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (LSTM+ BiLSTM + GARCH + Beta ):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_lstm_bi_garch_Beta')
          .head(3)[['Stock', 'Risk_Score_lstm_bi_garch_Beta']])



Safest stocks in IT (LSTM+ BiLSTM + GARCH + Beta ):
        Stock  Risk_Score_lstm_bi_garch_Beta
0  HCLTECH.NS                      -0.042929
1     INFY.NS                      -0.040542
2      TCS.NS                      -0.038039

Safest stocks in Banking (LSTM+ BiLSTM + GARCH + Beta ):
          Stock  Risk_Score_lstm_bi_garch_Beta
4   AXISBANK.NS                      -0.046655
6  ICICIBANK.NS                      -0.037657
7  KOTAKBANK.NS                      -0.035293

Safest stocks in Pharma (LSTM+ BiLSTM + GARCH + Beta ):
           Stock  Risk_Score_lstm_bi_garch_Beta
9     DRREDDY.NS                      -0.051259
10  SUNPHARMA.NS                      -0.049467
8       CIPLA.NS                      -0.043016

Safest stocks in Auto (LSTM+ BiLSTM + GARCH + Beta ):
            Stock  Risk_Score_lstm_bi_garch_Beta
13  TATAMOTORS.NS                      -0.059482
11         M&M.NS                      -0.042021
12      MARUTI.NS                      -0.038072

Safest stocks in FMC

GARCH AND BETA + GRU + LSTM

In [62]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [  0.01639802  -2.77465893   7.54076869 -13.68621258]
Intercept: 0.06700001873206124


In [63]:
features_df['Risk_Score_GRU_LSTM_GARCH_Beta'] = X @ weights


In [64]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta + GRU + LSTM):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_LSTM_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_GRU_LSTM_GARCH_Beta']])



Safest stocks in IT (GARCH + Beta + GRU + LSTM):
        Stock  Risk_Score_GRU_LSTM_GARCH_Beta
0  HCLTECH.NS                       -0.042905
2      TCS.NS                       -0.041395
1     INFY.NS                       -0.039796

Safest stocks in Banking (GARCH + Beta + GRU + LSTM):
          Stock  Risk_Score_GRU_LSTM_GARCH_Beta
4   AXISBANK.NS                       -0.044873
6  ICICIBANK.NS                       -0.037768
7  KOTAKBANK.NS                       -0.034003

Safest stocks in Pharma (GARCH + Beta + GRU + LSTM):
           Stock  Risk_Score_GRU_LSTM_GARCH_Beta
10  SUNPHARMA.NS                       -0.049863
9     DRREDDY.NS                       -0.049455
8       CIPLA.NS                       -0.042737

Safest stocks in Auto (GARCH + Beta + GRU + LSTM):
            Stock  Risk_Score_GRU_LSTM_GARCH_Beta
13  TATAMOTORS.NS                       -0.060673
11         M&M.NS                       -0.041959
12      MARUTI.NS                       -0.038221

Safest stocks in

In [65]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_lstm_bi_garch_Beta', 'Risk_Score_GRU_Bi_Beta_GARCH', 'Risk_Score_GRU_LSTM_GARCH_Beta']].head()


,Stock,Risk_Score,Risk_Score_lstm_bi_garch_Beta,Risk_Score_GRU_Bi_Beta_GARCH,Risk_Score_GRU_LSTM_GARCH_Beta
0,HCLTECH.NS,-0.042905,-0.042929,-0.047495,-0.042905
1,INFY.NS,-0.039796,-0.040542,-0.043281,-0.039796
2,TCS.NS,-0.041395,-0.038039,-0.042827,-0.041395
3,WIPRO.NS,-0.037301,-0.037420,-0.044187,-0.037301
4,AXISBANK.NS,-0.044873,-0.046655,-0.049435,-0.044873


Comparison per sector

In [66]:
def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()

    models = ['Risk_Score', 'Risk_Score_lstm_bi_garch_Beta', 'Risk_Score_GRU_Bi_Beta_GARCH', 'Risk_Score_GRU_LSTM_GARCH_Beta']

    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        print(f"\n🔹 Top {top_n} safest stocks using {model}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])


In [67]:
compare_sector_models("IT", top_n=3)
compare_sector_models("Banking", top_n=3)
compare_sector_models("Pharma", top_n=3)



📊 Sector: IT

🔹 Top 3 safest stocks using Risk_Score:


,Stock,Risk_Score
0,HCLTECH.NS,-0.042905
2,TCS.NS,-0.041395
1,INFY.NS,-0.039796



🔹 Top 3 safest stocks using Risk_Score_lstm_bi_garch_Beta:


,Stock,Risk_Score_lstm_bi_garch_Beta
0,HCLTECH.NS,-0.042929
1,INFY.NS,-0.040542
2,TCS.NS,-0.038039



🔹 Top 3 safest stocks using Risk_Score_GRU_Bi_Beta_GARCH:


...

,Stock,Risk_Score_lstm_bi_garch_Beta
9,DRREDDY.NS,-0.051259
10,SUNPHARMA.NS,-0.049467
8,CIPLA.NS,-0.043016



🔹 Top 3 safest stocks using Risk_Score_GRU_Bi_Beta_GARCH:


,Stock,Risk_Score_GRU_Bi_Beta_GARCH
10,SUNPHARMA.NS,-0.052978
9,DRREDDY.NS,-0.052067
8,CIPLA.NS,-0.045685



🔹 Top 3 safest stocks using Risk_Score_GRU_LSTM_GARCH_Beta:


,Stock,Risk_Score_GRU_LSTM_GARCH_Beta
10,SUNPHARMA.NS,-0.049863
9,DRREDDY.NS,-0.049455
8,CIPLA.NS,-0.042737


Comparison table

In [68]:
def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()

    for col in ['Risk_Score', 'Risk_Score_lstm_bi_garch_Beta', 'Risk_Score_GRU_Bi_Beta_GARCH', 'Risk_Score_GRU_LSTM_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')

    return ranking_df.sort_values('Risk_Score_Rank')


In [69]:
sector_ranking_table("FMCG")


,Stock,Risk_Score_Rank,Risk_Score_lstm_bi_garch_Beta_Rank,Risk_Score_GRU_Bi_Beta_GARCH_Rank,Risk_Score_GRU_LSTM_GARCH_Beta_Rank
16,ITC.NS,1.0,1.0,2.0,1.0
14,DABUR.NS,2.0,2.0,1.0,2.0
15,HINDUNILVR.NS,3.0,3.0,3.0,3.0
